In [ ]:
#libraries
%matplotlib notebook

import wavio
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from python_speech_features import mfcc
from IPython.core.display import clear_output

In [ ]:
sess = tf.Session()

#### Input Preprocessing
* d0.wav and d1.wav contain sound produced by the footsteps of two distinct people.
* Preprocessed using audacity to remove silence.
* Imported and converted to MFCC vectors using suitable fixed window. Tune window parameters for better performance.
* Concatenated into one training vector.
* Standardized to $ \mu = 0, \sigma = 1$.

In [ ]:
x0 = wavio.read('at7/d0.wav').data; x1 = wavio.read('at7/d1.wav').data
l0 = mfcc(x0[:,0],44100,0.2,0.1); l1 = mfcc(x1[:,0],44100,0.2,0.1)
fx = np.concatenate((l0,l1),0)
train_x = np.array(fx); mu = np.mean(train_x); sigma = np.std(train_x); #data standardization
train_x = (train_x - mu + 0.0)/sigma

num_classes = 2;
n_train = fx.shape[0]
n0 = l0.shape[0]
input_size = l0.shape[1]
x = tf.placeholder(tf.float32, [None, input_size])

#### Hyperparameters
* Adjust hidden_shape to add layers or neurons within the layers. Eg: [100,2,100] <=> 3 hidden layers with 100,2,100 neurons in the 1st,2nd,3rd layer respectively.
* Edit act_fns, code_lyr according to hidden_shape. Here the sparse representation is hidden layer no 2.

In [ ]:
hidden_shape = [100,2,100] #No of neurons in each hidden layer
act_fns = ['tanh','sig','tanh'] #Activation functions of layers
code_lyr = 2 #index of hidden layer containing sparse vector
num_lyr = len(hidden_shape) #No of hidden layers
net_shape = [input_size] + hidden_shape + [input_size]
learning_rate = tf.placeholder(tf.float32,[])
beta = tf.placeholder(tf.float32,[]) #sparsity_penalty weight

In [ ]:
#parameters
params = {'W':{},'b':{}} #W: Weights, b: biases
for lay_num in range(num_lyr+1):
    params['W'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num], net_shape[lay_num+1]],stddev = 0.5))
    params['b'][lay_num] = tf.Variable(tf.random_normal([net_shape[lay_num+1]],stddev = 0.5))

#### Network Graph
* Cost = Recontruction error + beta*Sparsity penalty
* Recontruction error is the L2 norm between the input and output (reconstructed) vector. This is usual for continuous input autoencoders
* The sparsity penalty is nonstandard. It is applied on the "sparse" 2-vector. Since it is output from a sigmoid layer, it lies in unit square centered at (0.5,0.5). A decreasing function of the distance from the line y=x is used. This ensures minima that are furthest apart in 2D space [At (1,0) and (0,1)]. Modify this function according to the output space of the 2-vector layer.
* The intuition behind the setup is that if the input contains MFCCs from two distinct people (i.e feature values from distinct classes), they should settle at the two distinct minima. This is because settling farther away from each other allows greater freedom in representational power for reconstruction. 
* This also allows use to deterministically say where the clusters will converge. One class should converge above the line y = x and another, below.

In [ ]:
#network graph
def act_fn(a,lay_num):
    if act_fns[lay_num] == 'tanh':
        return tf.tanh(a)
    elif act_fns[lay_num] == 'relu':
        return tf.nn.relu(a)
    elif act_fns[lay_num] == 'elu':
        return tf.nn.elu(a)
    elif act_fns[lay_num] == 'sig':
        return tf.sigmoid(a)
    
def AutoEncoder(x,params):
    z = x
    for lay_num in range(num_lyr):
        z = act_fn(tf.add(tf.matmul(z,params['W'][lay_num]),params['b'][lay_num]),lay_num)
        if lay_num == code_lyr-1:
            corner_pusher = -tf.reduce_mean(tf.abs(tf.sub(z[:,0],z[:,1])))/np.sqrt(2) #distance from line y = x
            sparsity_penalty = corner_pusher
            code = z
    x_ = tf.add(tf.matmul(z,params['W'][lay_num+1]),params['b'][lay_num+1])
    cost = tf.sqrt(tf.reduce_mean(tf.squared_difference(x,x_))) + tf.mul(beta,sparsity_penalty)
    return (cost,code)

#### Graph Construction
* Ordinary gradient descent optimization is used. Other optimizers may be considered to automate the learning process better.

In [ ]:
cost,code = AutoEncoder(x,params)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
#parameter initialization
sess.run(tf.initialize_all_variables())

#### Learning
* The model should have sufficient representational power:
    * The size of the sparse vector should scale with the number of clusters (and hence classes) we want to detect. Here num_classes = 2. Size(sparse vector) = 2 proved to be sufficient.
    * Set beta = 0 and adjust the size of other layers until the reconstruction error is low.
* Beta should be small enough that it doesn't substantitally affect the recontructive capability but still pushes the points to the desired peripheries.

In [ ]:
#learning
n_iter = 0
while True:
    try:
        _,cost_,code_ = sess.run([optimizer,cost,code],feed_dict={x:train_x,learning_rate:0.1,beta:0.05})
        if n_iter % 1000 == 0:
            print('iter'+str(n_iter)+' cost:'+str(cost_))
        if n_iter+1 % 20000 == 0:
            clear_output()
        n_iter += 1
    except KeyboardInterrupt:
        print "Training is stopped"
        break
sess.close()

In [ ]:
#plots
plt.title('Sparse Vectors'); plt.xlabel('Dimension 1'); plt.ylabel('Dimension 2');
plt.scatter(code_[n0:][:,0],code_[n0:][:,1],c=[1,0,0], label = 'Person 1')
plt.scatter(code_[:n0][:,0],code_[:n0][:,1],c=[0,0,1], label = 'Person 2')
plt.legend()
plt.plot([0,1],[0,1],c=[0,0,0],linewidth=2.0)
plt.show()

##### Prediction Confidence and Classification
* Each sparse vector (code_[i] where i is the index) is classified as the red class if y > x and blue class otherwise.
* Prediction confidence (after seeing n samples)  = $\dfrac{No\  of\  correctly\  classified\  points\ upto\ now}{n}$. If this number is over 0.5 i.e the clustering algorithm thinks it is more likely that the points belong to the correct class, we make the right prediction.
* If we want higher confidence estimates, especially when there is sizeable overlap betwwen the two cluster near the boundary (line y = x), We may want to throw away points close to the boundary. They don't help us discriminate much as they may be generated by noise or are low confidence signal points. An alternate definition of prediction confidence is then required: $ Prediction\ confidence[n]  =\dfrac{No\  of\  correctly\  classified\  points - No\  of\  misclassified\  points}{n}$. In this case, we require that prediction confidence > 0 (as opposed to 0.5).

In [ ]:
np.set_printoptions(threshold=np.inf)
np.cumsum((code_[n0:,0]< code_[n0:,1])+0.0)/(range(1,l1.shape[0]+1))

In [ ]:
np.cumsum((code_[:n0,0] > code_[:n0,1])+0.0)/(range(1,l0.shape[0]+1))